## Work Notebook
### Algorithm of Hybrid Rec Engine - find similar products according a tag classifier followed by cosine similarity by Spacy embeddings of reviewText 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

In [91]:
#loading downloaded data
df=pd.read_json('~/Downloads/reviews_Electronics_5.json', lines=True)

  0%|                             | 179/63001 [41:00:30<14392:20:12, 824.75s/it]


### Loading pickled webscraping files

In [3]:
with open('df.pickle', 'rb') as file:
    df = pickle.load(file)

### We will train a model that classifies user input (item) into appropriate tag labels 

In [92]:
df

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,AO94DHGC771SJ,0528881469,amazdnu,"[0, 0]",We got this GPS for my husband who is an (OTR)...,5,Gotta have GPS!,1370131200,"06 2, 2013"
1,AMO214LNFCEI4,0528881469,Amazon Customer,"[12, 15]","I'm a professional OTR truck driver, and I bou...",1,Very Disappointed,1290643200,"11 25, 2010"
2,A3N7T0DY83Y4IG,0528881469,C. A. Freeman,"[43, 45]","Well, what can I say. I've had this unit in m...",3,1st impression,1283990400,"09 9, 2010"
3,A1H8PY3QHMQQA0,0528881469,"Dave M. Shaw ""mack dave""","[9, 10]","Not going to write a long review, even thought...",2,"Great grafics, POOR GPS",1290556800,"11 24, 2010"
4,A24EV6RXELQZ63,0528881469,Wayne Smith,"[0, 0]",I've had mine for a year and here's what we go...,1,"Major issues, only excuses for support",1317254400,"09 29, 2011"
...,...,...,...,...,...,...,...,...,...
1689183,A34BZM6S9L7QI4,B00LGQ6HL8,"Candy Cane ""Is it just me?""","[1, 1]",Burned these in before listening to them for a...,5,Boom -- Pop -- Pow. These deliver.,1405555200,"07 17, 2014"
1689184,A1G650TTTHEAL5,B00LGQ6HL8,"Charles Spanky ""Zumina Reviews""","[0, 0]",Some people like DJ style headphones or earbud...,5,"Thin and light, without compromising on sound ...",1405382400,"07 15, 2014"
1689185,A25C2M3QF9G7OQ,B00LGQ6HL8,Comdet,"[0, 0]",I&#8217;m a big fan of the Brainwavz S1 (actua...,5,Same form factor and durability as the S1 with...,1405555200,"07 17, 2014"
1689186,A1E1LEVQ9VQNK,B00LGQ6HL8,J. Chambers,"[0, 0]","I've used theBrainwavz S1 In Ear Headphones, a...",5,Superb audio quality in a very comfortable set...,1405641600,"07 18, 2014"


### We will find the sentiment score fro each review 

In [37]:
df.shape

(63001, 9)

In [53]:
df.reviewText

0          0    We got this GPS for my husband who is an ...
5          5    I am using this with a Nook HD+. It works...
10         10    This item is just as was described in th...
18         18     This is a great buy, compared to a $60 ...
237        237    The Nook Tablet, in both the 16gb versi...
                                 ...                        
1689106    1689106    The Sabrent CR-CCU3 Card reader is ...
1689125    1689125    Great hub for carrying around if yo...
1689142    1689142    This is an interesting device in th...
1689162    1689162    ...all part of the package that com...
1689183    1689183    Burned these in before listening to...
Name: reviewText, Length: 63001, dtype: object

In [71]:
import nltk
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

In [95]:
sent=[sid.polarity_scores(x) for x in tqdm(df.reviewText)]

100%|████████████████████████████████| 1689188/1689188 [31:06<00:00, 904.93it/s]


In [110]:
df['vader_score']=[sent[x].get('compound') for x in range(len(sent))]

In [113]:
# Mapped Vader scores to integer counts for easier manipulation 
col         = 'vader_score'
conditions  = [ df[col] >= .15, (df[col] < .1) & (df[col]> -.1), df[col] <= -.1 ]
choices     = [ 1, 0, -1 ]
df["vader_score"] = np.select(conditions, choices, default=np.nan)  

In [115]:
#final mapping process
df['vader_score']= df['vader_score'].map({-1:'negative', 0: 'neutral', 1:'positive'})

In [117]:
df.vader_score.value_counts()

positive    1399723
negative     194525
neutral       80603
Name: vader_score, dtype: int64

### Then we combine the two inputs 

In [ ]:
model=Models.Sequential()
model1 = Sequential()

model2 = Model(inputs=[model.input, model1.input], outputs=x)